In [ ]:
import torchvision
import torch
from PIL import Image


# Create a vector of zeros of size 5
size = (128, 128)
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(size), torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.Flowers102(
    "../Module 1/flowers", "train", transform=transform, download=True)
test_dataset = torchvision.datasets.Flowers102(
    "../Module 1/flowers", "test", transform=transform, download=True)


In [3]:
train_dataset

Dataset Flowers102
    Number of datapoints: 1020
    Root location: ../Module 1/flowers
    split=train
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [7]:
len(train_dataset)

1020

In [6]:
train_dataset[0]

(tensor([[[0.0471, 0.0706, 0.0745,  ..., 0.1255, 0.4667, 0.5647],
          [0.0667, 0.0667, 0.0549,  ..., 0.1333, 0.4824, 0.5647],
          [0.0824, 0.0745, 0.0549,  ..., 0.1451, 0.5059, 0.5686],
          ...,
          [0.1059, 0.1059, 0.0863,  ..., 0.5020, 0.4902, 0.4706],
          [0.1137, 0.1137, 0.1294,  ..., 0.5059, 0.4784, 0.4706],
          [0.1020, 0.1176, 0.1176,  ..., 0.5020, 0.4745, 0.4667]],
 
         [[0.0863, 0.1255, 0.1373,  ..., 0.1294, 0.3412, 0.3961],
          [0.0941, 0.1098, 0.1059,  ..., 0.1294, 0.3490, 0.3922],
          [0.0941, 0.0941, 0.0824,  ..., 0.1294, 0.3608, 0.3843],
          ...,
          [0.2000, 0.1804, 0.1333,  ..., 0.4235, 0.4118, 0.3922],
          [0.2118, 0.2039, 0.2000,  ..., 0.4275, 0.4039, 0.3922],
          [0.2078, 0.2196, 0.2196,  ..., 0.4196, 0.4078, 0.3765]],
 
         [[0.0314, 0.0392, 0.0353,  ..., 0.0863, 0.4745, 0.5961],
          [0.0392, 0.0353, 0.0235,  ..., 0.0980, 0.4902, 0.5922],
          [0.0431, 0.0353, 0.0235,  ...,

In [8]:
help(torch.utils.data.DataLoader)

Help on class DataLoader in module torch.utils.data.dataloader:

class DataLoader(typing.Generic)
 |  DataLoader(dataset: torch.utils.data.dataset.Dataset[+_T_co], batch_size: Optional[int] = 1, shuffle: Optional[bool] = None, sampler: Union[torch.utils.data.sampler.Sampler, Iterable, NoneType] = None, batch_sampler: Union[torch.utils.data.sampler.Sampler[List], Iterable[List], NoneType] = None, num_workers: int = 0, collate_fn: Optional[Callable[[List[~_T]], Any]] = None, pin_memory: bool = False, drop_last: bool = False, timeout: float = 0, worker_init_fn: Optional[Callable[[int], NoneType]] = None, multiprocessing_context=None, generator=None, *, prefetch_factor: Optional[int] = None, persistent_workers: bool = False, pin_memory_device: str = '', in_order: bool = True)
 |
 |  Data loader combines a dataset and a sampler, and provides an iterable over the given dataset.
 |
 |  The :class:`~torch.utils.data.DataLoader` supports both map-style and
 |  iterable-style datasets with singl

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self, layer_size = [512, 512, 512]) -> None:
        super(MyModel, self).__init__()
        layers = []
        layers.append(torch.nn.Flatten())
        c = 128*128*3 # number of channel
        for s in layer_size:
            layers.append(torch.nn.Linear(c, s)) # each alayer go from c to s (from previous dimension to new dimension)
            layers.append(torch.nn.ReLU())
            c = s
        layers.append(torch.nn.Linear(c, 102))
        # initially we have a linear layer goes from 128*128*3 to 512
        # then c = 512 then we go from 512 to 512
        # then we go from 512 to 512 again
        # then we goes from c to 102 which is our output
        # the ReLU add between the last linear and the linear in the for loop
        self.model = torch.nn.Sequential(*layers)
        # the sequential will create a network where the layer is called first
        # then the next layer is called the next layer and then everything in the for loop is called
        # in the end linear layer is called to 102 outputs
        # sequential does is create a network where the output from one layer is fed into it as an input to the next layer until we don't have anything anymore

    def forward(self, x):
        return self.model(x)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

model = MyModel(layer_size=[])
model.cuda()

loss_fn = torch.nn.CrossEntropyLoss()
    
optim = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

for epoch in range(10): 
    for imgs, labels in train_loader:
        imgs, labels = imgs.cuda(), labels.cuda()
        pred = model(imgs)

        loss_val = loss_fn(pred, labels)

        optim.zero_grad()
        loss_val.backward()
        optim.step()

        print(loss_val.item())


4.609244346618652
4.691183090209961
4.666364669799805
4.642425537109375
4.688011646270752
4.725284576416016
4.610206127166748
4.642334461212158
4.662638187408447
4.690561771392822
4.647457599639893
4.659372806549072
4.686764717102051
4.669470310211182
4.64767599105835
4.638030052185059
4.56156063079834
4.6193037033081055
4.582309246063232
4.559229850769043
4.601150035858154
4.550866603851318
4.579258441925049
4.567873954772949
4.6033830642700195
4.556581020355225
4.573458194732666
4.561983108520508
4.640350818634033
4.4944939613342285
4.595992088317871
4.570327281951904
4.4774041175842285
4.521651268005371
4.5165605545043945
4.5041093826293945
4.496799945831299
4.506601810455322
4.475484371185303
4.4646897315979
4.495852947235107
4.490749359130859
4.479709148406982
4.492855548858643
4.514074802398682
4.480870246887207
4.547335147857666
4.46812105178833
4.340383529663086
4.389892101287842
4.452930450439453
4.393560409545898
4.387499809265137
4.441415786743164
4.405380725860596
4.3939595